In [22]:
from dotenv import load_dotenv

MODEL = "llama2"

In [23]:
from langchain_ollama import OllamaLLM,OllamaEmbeddings 
model = OllamaLLM(model=MODEL)
model.invoke("Tell me a joke")
embaddings = OllamaEmbeddings(model=MODEL)

In [24]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("Tell me a joke")

"\nWhy don't scientists trust atoms? Because they make up everything! 😂"

In [25]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("ADVIL.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'producer': 'www.adlibsoftware.com: CTP (5.10.0.7216) OS (Windows 2012,2,0,64)', 'creator': 'Microsoft Word(12.0)', 'creationdate': '2016-09-19T17:38:26-04:00', 'author': 'mazuren', 'moddate': '2021-09-29T13:31:53+05:30', 'r_object_id': '090177e189b0de81', 'source': 'ADVIL.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='Advil Tablets\nIbuprofen Tablets, 200 mg\nThis is the most current labeling information and may differ from labels on product packaging. If there are any \ndifferences between this website labeling and product packaging labeling, this website labeling should be regarded \nas the most current.\nLBL-00000027 (Version 38.0) Page 1 of 3\nDrug Facts\nActive ingredient (in each tablet)                                           Purpose\nIbuprofen 200 mg (NSAID)*   ………………………………….Pain reliever/Fever reducer\n*nonsteroidal anti-inflammatory drug\nUses\n\uf06e temporarily relieves minor aches and pains due to:\n\uf06e headache \n\uf06e toot

In [26]:
from langchain.prompts import PromptTemplate

template = """
        Answer the question based on the context below. If you can't
        answer the question, reply "I don't know".

        Context: {context}

        Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


        Answer the question based on the context below. If you can't
        answer the question, reply "I don't know".

        Context: Here is some context

        Question: Here is a question



In [27]:
chain = prompt | model | parser 

In [28]:
chain.input_schema.model_json_schema()

{'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}},
 'required': ['context', 'question'],
 'title': 'PromptInput',
 'type': 'object'}

In [29]:
chain.invoke(
    {
        "context": "The name I was given was Denisa",
        "question": "What's my name?"
    }
)

'Hi there! Based on the context you provided, your name is Denisa.'

In [30]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embaddings)

In [31]:
import pandas as pd

df = pd.DataFrame([d.page_content for d in pages], columns=["text"])

df.head(10)


,text
0,"Advil Tablets\nIbuprofen Tablets, 200 mg\nThis..."
1,"Advil Tablets\nIbuprofen Tablets, 200 mg\nThis..."
2,"Advil Tablets\nIbuprofen Tablets, 200 mg\nThis..."


In [41]:
from giskard.rag import KnowledgeBase

knowledge_base = KnowledgeBase(df)


ModuleNotFoundError: No module named 'numpy.strings'

In [ ]:
retriever = vectorstore.as_retriever()

retriever.invoke("ADVIL")

In [ ]:
from operator import itemgetter

chain =(

{
    "context": itemgetter("question") | retriever,
    "question": itemgetter("question")
} |

prompt | model | parser )

chain.invoke({
    "question":"What is advil?"
})

In [ ]:
questions = [
    "Can children under 12 years take advil?"
]

In [ ]:
for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question':question})}")
    print()